# Simple CNN with PyTorch


In [8]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms

## 1. Data Loader


A data loader can be treated as a list (or iterator, technically). Each time it will provide a minibatch of (img, label) pairs.


In [9]:
# Choose a dataset -- MNIST for example
dataset = datasets.MNIST(root='data', train=True, download=True)

# Set how the input images will be transformed
dataset.transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.1307,], std=[0.3081,])
])

# Create a data loader
train_loader = DataLoader(dataset, batch_size=256, shuffle=True, num_workers=1)

# Show the shape of a batch.

print("The shape of one batch is {}".format((next(iter(train_loader)))[0].size()))

The shape of one batch is torch.Size([256, 1, 28, 28])


## 2. Model


In [10]:

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()  # Call parent class's constructor
        
        self.conv1 = nn.Conv2d(1, 5, kernel_size=3, stride=2, padding=1)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(5, 8, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.fc = nn.Linear(8 * 7 * 7, 256)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(256, 10)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool(x)
        
        print(x.shape)
        x = x.view(-1, 8 * 7 * 7)
        x = self.fc(x)
        x = self.relu3(x)
        x = self.fc2(x)
        return x



model = SimpleCNN()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
#

SimpleCNN(
  (conv1): Conv2d(1, 5, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (relu1): ReLU()
  (conv2): Conv2d(5, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=392, out_features=256, bias=True)
  (relu3): ReLU()
  (fc2): Linear(in_features=256, out_features=10, bias=True)
)

## Loss and Optimizer


In [11]:

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)


## 4. Start training

The next step is to start the training process.

In [12]:
def train(epoch):
    model.train()  # Set the model to be in training mode
    for batch_index, (inputs, targets) in enumerate(train_loader):
        # Forward
        inputs = inputs.to(device)
        targets = targets.to(device)
        
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        if batch_index % 10 == 0:
            print('epoch {}  batch {}/{}  loss {:.3f}'.format(
                epoch, batch_index, len(train_loader), loss.item()))

        # Backward
        optimizer.zero_grad()  # Set parameter gradients to zero
        loss.backward()        # Compute (or accumulate, actually) parameter gradients
        optimizer.step()       # Update the parameters


In [13]:
# Choose a dataset -- MNIST
dataset = datasets.MNIST(root='data', train=False, download=True)

# Set how the input images will be transformed
dataset.transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.1307, ], std=[0.3081, ])
])

# Create a data loader
test_dataloader = DataLoader(dataset, batch_size=64, shuffle=False, num_workers=1)

def test(dataloader):
    model.eval()

    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to('cuda'), labels.to('cuda')
            outputs = model(images)
            _, predicted = torch.max(outputs.data, -1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Print the accuracy and loss
    accuracy = correct / total
    print('Accuracy:', accuracy)

In [14]:
for epoch in range(0, 6):
    train(epoch)
    test(test_dataloader)

torch.Size([256, 8, 7, 7])
epoch 0  batch 0/235  loss 2.300
torch.Size([256, 8, 7, 7])
torch.Size([256, 8, 7, 7])
torch.Size([256, 8, 7, 7])
torch.Size([256, 8, 7, 7])
torch.Size([256, 8, 7, 7])
torch.Size([256, 8, 7, 7])
torch.Size([256, 8, 7, 7])
torch.Size([256, 8, 7, 7])
torch.Size([256, 8, 7, 7])
torch.Size([256, 8, 7, 7])
epoch 0  batch 10/235  loss 2.299
torch.Size([256, 8, 7, 7])
torch.Size([256, 8, 7, 7])
torch.Size([256, 8, 7, 7])
torch.Size([256, 8, 7, 7])
torch.Size([256, 8, 7, 7])
torch.Size([256, 8, 7, 7])
torch.Size([256, 8, 7, 7])
torch.Size([256, 8, 7, 7])
torch.Size([256, 8, 7, 7])
torch.Size([256, 8, 7, 7])
epoch 0  batch 20/235  loss 2.295
torch.Size([256, 8, 7, 7])
torch.Size([256, 8, 7, 7])
torch.Size([256, 8, 7, 7])
torch.Size([256, 8, 7, 7])
torch.Size([256, 8, 7, 7])
torch.Size([256, 8, 7, 7])
torch.Size([256, 8, 7, 7])
torch.Size([256, 8, 7, 7])
torch.Size([256, 8, 7, 7])
torch.Size([256, 8, 7, 7])
epoch 0  batch 30/235  loss 2.288
torch.Size([256, 8, 7, 7])
t